In [5]:
from optimisation_ntn.simulation import Simulation, SimulationConfig
from optimisation_ntn.algorithms.assignment.qlearning_trainer import QLearningTrainer
import numpy as np
from optimisation_ntn.algorithms.assignment.strategy_factory import QLearningAssignment

np.random.seed(1)

# Create simulation
sim = Simulation(
    config=SimulationConfig(
        seed=None,
        debug=False,
        user_count=20,
        power_strategy="OnDemand",
        save_results=False,
    )
)

# Create trainer
trainer = QLearningTrainer(
    simulation=sim,
    episodes=100,
    save_path="qtable.pkl",
)

# Train the agent
trainer.train()

# Test the trained agent
print("\nTesting trained agent...")
sim = Simulation(
    config=SimulationConfig(
        seed=3,
        debug=False,
        user_count=20,
        power_strategy="OnDemand",
        save_results=False,
    )
)
sim.reset()
strategy = QLearningAssignment(sim.network, epsilon=0, qtable_path="qtable.pkl")
sim.assignment_strategy = strategy
sim.run()

print(f"Trained agent Final energy consumed: {sim.system_energy_consumed:.2f}")
print(
    f"Trained agent Final QoS satisfaction: {sim.evaluate_qos_satisfaction()/100:.2%}"
)

sim.reset()
strategy = QLearningAssignment(sim.network, epsilon=1.0, qtable_path="qtable.pkl")
sim.assignment_strategy = strategy
sim.run()

print(f"Random agent Final energy consumed: {sim.system_energy_consumed:.2f}")
print(f"Random agent Final QoS satisfaction: {sim.evaluate_qos_satisfaction()/100:.2%}")

Max energy: 4776.30, Min energy: 4364.90
Starting training...
QoS score: 0.95, Energy consumed: 3970.80
Energy score: 1.00
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4681.80
Energy score: 0.12
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4472.80
Energy score: 0.38
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4507.00
Energy score: 0.33
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4683.00
Energy score: 0.12
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4559.50
Energy score: 0.27
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4752.40
Energy score: 0.03
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4454.30
Energy score: 0.40
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consumed: 4572.80
Energy score: 0.25
Max energy: 4776.30, Min energy: 3970.80
QoS score: 0.9, Energy consu

In [2]:
from optimisation_ntn.simulation import Simulation, SimulationConfig
from optimisation_ntn.algorithms.assignment.matrix_based import MatrixBasedAssignment
import numpy as np

from optimisation_ntn.algorithms.assignment.strategy_factory import (
    QLearningAssignment,
    RandomAssignment,
    TimeGreedyAssignment,
    ClosestNodeAssignment,
    HAPSOnlyAssignment,
)

sim = Simulation(
    config=SimulationConfig(
        seed=10,
        user_count=10,
        power_strategy="OnDemand",
        save_results=False,
    )
)

assignment_vector = np.array([0, 0, 3, 0, 4, 4, 4, 4, 4, 3])
assignment_vector = np.array([4, 0, 2, 1])
matrix_strategy = MatrixBasedAssignment(sim.network)
matrix_strategy.set_assignment_matrix(assignment_vector)
sim.assignment_strategy = matrix_strategy

while sim.current_time < sim.max_time:
    sim.step()

energy_consumed = sim.system_energy_consumed
print(energy_consumed)
print(sim.evaluate_qos_satisfaction())

793.5990524629937
40.0


In [3]:
sim.reset()
assignment_vector = np.array([0, 0, 3, 0, 4, 4, 4, 4, 4, 3])
assignment_vector = np.array([0, 3, 3, 0, 0, 1, 2, 0, 4, 2])
matrix_strategy = MatrixBasedAssignment(sim.network)
matrix_strategy.set_assignment_matrix(assignment_vector)
sim.assignment_strategy = matrix_strategy

while sim.current_time < sim.max_time:
    sim.step()

energy_consumed = sim.system_energy_consumed
print(energy_consumed)
print(sim.evaluate_qos_satisfaction())

1764.498104925987
90.0


In [4]:
sim.reset()
strategy_instance = QLearningAssignment(sim.network)
strategy_instance.epsilon = 0
sim.assignment_strategy = strategy_instance
sim.assignment_strategy.load_q_table("../qtable.npy")
sim.run()

AttributeError: 'QLearningAssignment' object has no attribute 'load_q_table'

In [ ]:
sim.reset()
sim.run_with_assignment(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

(1133.3976311574843, 0.5)

In [ ]:
sim.reset()
sim.assignment_strategy = QLearningAssignment(sim.network)
sim.run()

energy_consumed = sim.system_energy_consumed
qos_score = sim.evaluate_qos_satisfaction()
print(energy_consumed)
print(qos_score)

# Load and analyze Q-table
q_learning = QLearningAssignment(sim.network)
q_learning.load_q_table("../qtable.npy")

# Print summary
q_learning.print_q_table_summary()

# Interpret specific state
state = list(q_learning.q_table.keys())[0]  # Get first state
print("\nAnalyzing first state:")
print(q_learning.interpret_state(state))

1784.6971573889812
80.0
Q-table contains 13 states

Best learned action:
State interpretation:

Node Energy Levels:
Node 0: High
Node 1: Infinite
Node 2: Infinite
Node 3: Infinite
Node 4: High
Node 5: Infinite

Node States:
Node 0: Off
Node 1: Off
Node 2: Off
Node 3: Off
Node 4: On

Request Size: ≤6MB

Best action: Node 0
Q-value: 0.0000

Analyzing first state:
State interpretation:

Node Energy Levels:
Node 0: High
Node 1: Infinite
Node 2: Infinite
Node 3: Infinite
Node 4: Infinite
Node 5: High

Node States:
Node 0: Off
Node 1: Off
Node 2: Off
Node 3: Off
Node 4: Off
Node 5: Off

Request Size: ≤6MB


In [4]:
strategies = [
    QLearningAssignment,
    RandomAssignment,
    TimeGreedyAssignment,
    ClosestNodeAssignment,
    HAPSOnlyAssignment,
]

sim = Simulation(
    config=SimulationConfig(
        seed=1,
        user_count=10,
        power_strategy="OnDemand",
        save_results=False,
    )
)

strategy_instance = RandomAssignment(sim.network)
sim.assignment_strategy = strategy_instance
sim.run()
energy_consumed = sim.system_energy_consumed
qos_score = sim.evaluate_qos_satisfaction()

for strategy in strategies:
    sim.reset()
    strategy_instance = strategy(sim.network)
    sim.assignment_strategy = strategy_instance
    sim.run()
    energy_consumed = sim.system_energy_consumed
    qos_score = sim.evaluate_qos_satisfaction()
    print("-" * 10)
    print(strategy.__name__)
    print(energy_consumed)
    print(qos_score)

----------
QLearningAssignment
1963.8971573889812
100.0
----------
RandomAssignment
2055.7966836204782
100.0
----------
TimeGreedyAssignment
2673.896683620478
90.0
----------
ClosestNodeAssignment
1761.8000000000002
100.0
----------
HAPSOnlyAssignment
1885.0
100.0


In [3]:
# do a brute force search for the best assignment matrix

import itertools
from optimisation_ntn.algorithms.assignment.strategy_factory import (
    AssignmentStrategyFactory,
)
from optimisation_ntn.simulation import Simulation, SimulationConfig
import numpy as np

sim = Simulation(
    config=SimulationConfig(
        seed=1,
        user_count=2,
        power_strategy="OnDemand",
        save_results=False,
    )
)


n_nodes = len(sim.network.compute_nodes)
n_users = len(sim.network.user_nodes)

all_matrices = list(itertools.product(range(n_nodes), repeat=n_users))
print(len(all_matrices))

min_energy = float("inf")
best_matrix = None
# run all the combinations of assignment matrices (n_nodes^n_users)
for i in range(len(all_matrices)):
    assignment_matrix = all_matrices[i]
    sim.reset()
    sim.run_with_assignment(assignment_matrix)
    energy_consumed = sim.system_energy_consumed
    qos_score = sim.evaluate_qos_satisfaction()
    # print(energy_consumed)
    # print(qos_score)
    if qos_score == 100:
        if energy_consumed < min_energy:
            print(f"New best matrix found! Energy: {energy_consumed}, QoS: {qos_score}")
            min_energy = energy_consumed
            best_matrix = assignment_matrix

    if i % 100 == 0:
        print(f"Processed {i/len(all_matrices)*100:.2f}%")

print(best_matrix)
print(min_energy)

ModuleNotFoundError: No module named 'optimisation_ntn.algorithms.assignment.qlearning'